# ML.NET Anomaly Detection 

In [ ]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"
#r "nuget:XPlot.Plotly"
#r "nuget:XPlot.GoogleCharts"

Installed Packages Microsoft.ML, 1.7.0 Microsoft.ML.TimeSeries, 1.7.0 XPlot.GoogleCharts, 3.0.1 XPlot.Plotly, 4.0.6

In [ ]:
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Transforms.TimeSeries
open System.IO
open System.Linq
open System.Collections.Generic

## Following The Tutorial From [Here](https://docs.microsoft.com/en-us/dotnet/machine-learning/tutorials/sales-anomaly-detection)

In [ ]:
type ProductSalesData() =
    [<DefaultValue>]
    [<LoadColumn(0)>]
    val mutable public Month : string

    [<DefaultValue>]
    [<LoadColumn(1)>]
    val mutable public numSales : float32

type ProductSalesPrediction() = 
    [<DefaultValue>]
    [<VectorType(3)>]
    val mutable public Prediction : double[]

let dataPath : string = Path.Combine( __SOURCE_DIRECTORY__, "Data", "product-sales.csv")
let docSize : int = 36

In [ ]:
let ctx = MLContext()

let dataView = 
  ctx
    .Data
    .LoadFromTextFile<ProductSalesData>(
      path = dataPath,
      hasHeader = true,
      separatorChar = ',')

let anomalyPValueHistoryLength = 30
let changePointPValueHistoryLength = 10
let anomalyConfidence = 95
let changePointConfidence = 95

let anomalyPipeline =
    ctx.Transforms.DetectIidSpike(
      outputColumnName = "Prediction",
      inputColumnName = "numSales",
      side = AnomalySide.TwoSided,
      confidence = anomalyConfidence, 
      pvalueHistoryLength = anomalyPValueHistoryLength)

let trainedAnomalyModel = anomalyPipeline.Fit(dataView)
let transformedAnomalyData = trainedAnomalyModel.Transform(dataView);

let anomalies = 
    ctx.Data.CreateEnumerable<ProductSalesPrediction>(transformedAnomalyData, reuseRowObject = false)

let anomalyChartData = 
  anomalies
  |> Seq.map (fun p -> (p.Prediction.[0], p.Prediction.[1]))
  |> List.ofSeq 

In [ ]:
display(anomalyChartData)

index,Item1,Item2
0,0,271
1,0,150.89999389648438
2,0,188.10000610351562
3,0,124.30000305175781
4,0,185.3000030517578
5,0,173.5
6,0,236.8000030517578
7,0,229.5
8,0,197.8000030517578
9,0,127.9000015258789
